# Building LLM applications: Notebook 01

# Langchain basics

## Initialize

In [1]:
import os
import dotenv

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [2]:
MODEL = 'llama3.2:3b-instruct-fp16'

In [ ]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

## Excercise 01: Connect to an LLM

Connect to an LLM and make a simple request (e..g translate a senetence)

```
Translate to French: I like coffee.
```

Take a look at the LLM parameters, and full response 

In [ ]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

messages = "Translate to French: I like coffee."

In [ ]:
# Invoke the LLM and show the result

# ADD YOUR CODE HERE
#
#

## Excercise 02: Prompt template

In [ ]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)

In [13]:
# Create a "chain" of runnable object (prompt and LLM), invoke it and show the result

# ADD YOUR CODE HERE
#
#
